# ResNet Predictions

In [56]:
# Packages
import torch
from torchvision import models
import torch.nn as nn
import os
import pandas as pd
from torchvision import transforms
from PIL import Image

## Load Test Data

In [57]:
# Load data
directory_path = '../../../Data/Features/All Features/test/'
# list of files in directory
file_list = [f for f in os.listdir(directory_path) if f.endswith('.parquet')]
# read in all parquet files
combined_df = pd.concat([pd.read_parquet(directory_path + f, columns=['Class', 'harmonized_filename', 'image_path_blur', 'image_path_no_blur']) for f in file_list])
# replace .jpg with _no_blur.jpg
combined_df['harmonized_filename'] = combined_df['harmonized_filename'].str.replace('.jpg', '_no_blur.jpg')
# reset index
combined_df.reset_index(drop=True, inplace=True)
combined_df.head()

,Class,harmonized_filename,image_path_blur,image_path_no_blur
0,Pickup,Pickup_test_orig_test_05620_resized_no_blur.jpg,../../../Images/test/Blurred/Pickup_test_orig_...,../../../Images/test/No_Blur/Pickup_test_orig_...
1,SUV,SUV_test_orig_test_08041_resized_no_blur.jpg,../../../Images/test/Blurred/SUV_test_orig_tes...,../../../Images/test/No_Blur/SUV_test_orig_tes...
2,Convertible,Convertible_test_orig_test_00822_resized_no_bl...,../../../Images/test/Blurred/Convertible_test_...,../../../Images/test/No_Blur/Convertible_test_...
3,Convertible,Convertible_test_orig_test_00618_resized_no_bl...,../../../Images/test/Blurred/Convertible_test_...,../../../Images/test/No_Blur/Convertible_test_...
4,Pickup,Pickup_test_orig_test_05241_resized_no_blur.jpg,../../../Images/test/Blurred/Pickup_test_orig_...,../../../Images/test/No_Blur/Pickup_test_orig_...


In [58]:
# Standard ResNet preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## Load Model

In [63]:
# Load model at '../../../Output/Classifier Fitting/ResNet/resnet.pth'
# Initialize ResNet50
model = models.resnet50(pretrained=True)
# Modify the final layer for our classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(combined_df['Class'].unique()))
# Load state
model.load_state_dict(torch.load('../../../Output/Classifier Fitting/ResNet/resnet.pth', map_location=torch.device('cpu')))
# Put model in evaluation mode
model.eval()
model

c:\Users\ijyli\miniforge3\envs\computervision\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ijyli\miniforge3\envs\computervision\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Perform Inference

In [64]:
def get_image_prediction(image_path, model, processor):
    # Load image
    image = Image.open(image_path).convert("RGB")
    # Pre-process image
    preprocessed_image = processor(image)
    # Run the image through the model
    # Add batch dimension
    with torch.no_grad():
        outputs = model(torch.unsqueeze(preprocessed_image, 0))
    return outputs

# Get predictions
image_paths = ['../../../Images/test/No Blur/' + hf for hf in combined_df['harmonized_filename']]
predictions = [get_image_prediction(image_path, model, preprocess) for image_path in image_paths]
print(predictions[0])

tensor([[-2.2226, -6.4006,  8.7149, -0.8922]])


## Save Predictions

In [65]:
# Save predictions
# Create dataframe
predictions_df = combined_df.copy()
# Add predictions
predictions_df['ResNet_predictions'] = [torch.argmax(p).item() for p in predictions]
# Load label mapping
label_mapping = pd.read_excel('../../../Output/Classifier Fitting/ResNet/label_mapping.xlsx')
# Convert to dictionary
label_mapping = label_mapping.to_dict()['Class']
print(label_mapping)
# Map predictions to labels
predictions_df['ResNet_predictions'] = predictions_df['ResNet_predictions'].map(label_mapping)
# Save to Excel
predictions_df.to_excel('../../../Data/Predictions/ResNet/ResNet_predictions.xlsx', index=False)

{0: 'Sedan', 1: 'Convertible', 2: 'Pickup', 3: 'SUV'}


In [66]:
# Value counts of ResNet predictions
predictions_df['ResNet_predictions'].value_counts()

ResNet_predictions
Sedan          598
SUV            449
Convertible    268
Pickup         174
Name: count, dtype: int64